In [1]:
import sys
sys.path.append('../src')

In [2]:
# import numpy as np

# class ARNonstationaryGenerator:
#     """Generator for non-stationary time series using AR process with varying weights"""

#     def __init__(self, seq_length=900, num_samples=1000, seed=42, noise_variance=16):
#         np.random.seed(seed)
#         self.seq_length = seq_length
#         self.num_samples = num_samples
#         self.noise_variance = noise_variance

#         # Default segment points and AR coefficients
#         self.segment_points = [0, 300, 600, 900]
#         self.ar_coefficients = [
#             [0.8, -0.5],  # First segment (Stable)
#             [0.6, -0.2],  # Second segment (Stable)
#             [0.4, -0.3],  # Third segment (Stable)
#         ]

#     def generate_ar_sequence(self):
#         """
#         Generate a non-stationary signal with varying AR coefficients.

#         Returns:
#             np.ndarray: Generated non-stationary signal.
#         """
#         seq_length = self.seq_length
#         # Initialize signal and input
#         x = np.random.normal(0, 1, seq_length)
#         q = np.random.normal(0, np.sqrt(self.noise_variance), seq_length)
#         y = np.zeros(seq_length)

#         # Generate signal for each segment
#         for seg_idx in range(len(self.segment_points) - 1):
#             start = self.segment_points[seg_idx]
#             end = self.segment_points[seg_idx + 1]
#             w = self.ar_coefficients[seg_idx]

#             for n in range(start, end):
#                 if n >= 2:
#                     y[n] = w[0] * x[n] + w[1] * x[n - 1] + q[n]
#                 elif n == 1:
#                     y[n] = w[0] * x[n] + q[n]
#                 else:  # n == 0
#                     y[n] = q[n]

#         return y

#     def generate(self):
#         """Generate non-stationary time series with varying AR coefficients and time features"""
#         data = np.zeros((self.num_samples, self.seq_length))

#         for i in range(self.num_samples):
#             # Generate each sample using the AR sequence
#             data[i] = self.generate_ar_sequence()

#         # Generate time features for each sample
#         # For simplicity, we'll use sine and cosine of normalized time index
#         time_index = np.arange(self.seq_length) / self.seq_length
#         time_features = np.stack([
#             np.sin(2 * np.pi * time_index),
#             np.cos(2 * np.pi * time_index)
#         ], axis=1)  # Shape: (seq_length, 2)
#         time_features = np.tile(time_features, (self.num_samples, 1, 1))  # Shape: (num_samples, seq_length, 2)

#         return data, time_features


In [3]:
from torch.utils.data import Dataset

class SyntheticDataset(Dataset):
    def __init__(self, data, time_features, seq_len, label_len, pred_len):
        self.data = data  # Shape: (num_samples, seq_length)
        self.time_features = time_features  # Shape: (num_samples, seq_length, time_feature_dim)
        self.seq_len = seq_len
        self.label_len = label_len
        self.pred_len = pred_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        ts = self.data[idx]  # Shape: (seq_length,)
        tf = self.time_features[idx]  # Shape: (seq_length, time_feature_dim)

        # Define the indices for encoder and decoder
        total_len = self.seq_len + self.pred_len
        if len(ts) < total_len:
            # Handle cases where the time series is too short
            raise IndexError("Time series too short for the given sequence length.")

        # Encoder inputs
        x_enc = ts[:self.seq_len]
        x_mark_enc = tf[:self.seq_len]

        # Decoder inputs
        x_dec = ts[self.seq_len - self.label_len:self.seq_len + self.pred_len]
        x_mark_dec = tf[self.seq_len - self.label_len:self.seq_len + self.pred_len]

        # Targets
        y = ts[self.seq_len:self.seq_len + self.pred_len]

        # Add feature dimension if necessary
        x_enc = x_enc.reshape(-1, 1)
        x_dec = x_dec.reshape(-1, 1)
        y = y.reshape(-1, 1)

        # Convert to torch tensors
        x_enc = torch.tensor(x_enc, dtype=torch.float32)
        x_mark_enc = torch.tensor(x_mark_enc, dtype=torch.float32)
        x_dec = torch.tensor(x_dec, dtype=torch.float32)
        x_mark_dec = torch.tensor(x_mark_dec, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)

        return x_enc, x_mark_enc, x_dec, x_mark_dec, y


In [4]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn

from models.timesnet.lora_timesnet import LoRATimesNet


class LoRAFineTuner:
    def __init__(
        self,
        base_model,
        rank=4,
        alpha=1.0,
        learning_rate=1e-3,
        batch_size=32,
        num_epochs=10,
    ):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.lora_model = LoRATimesNet(base_model, rank, alpha).to(self.device)
        self.optimizer = optim.AdamW(
            self.lora_model.get_lora_parameters(), lr=learning_rate
        )
        self.batch_size = batch_size
        self.num_epochs = num_epochs

    def train_on_synthetic(self, synthetic_generator, num_samples=1000):
        """Fine-tune on synthetic data"""
        # Generate synthetic data
        synthetic_data, time_features = synthetic_generator.generate()

        # Create dataset
        dataset = SyntheticDataset(
            data=synthetic_data,
            time_features=time_features,
            seq_len=self.lora_model.base_model.configs.seq_len,
            label_len=self.lora_model.base_model.configs.label_len,
            pred_len=self.lora_model.base_model.configs.pred_len
        )

        # Create dataloader
        train_loader = DataLoader(
            dataset, batch_size=self.batch_size, shuffle=True
        )

        # Training loop
        self.lora_model.train()
        for epoch in range(self.num_epochs):
            total_loss = 0
            for x_enc, x_mark_enc, x_dec, x_mark_dec, y in train_loader:
                self.optimizer.zero_grad()

                x_enc = x_enc.to(self.device)  # Shape: [batch_size, seq_len, 1]
                x_mark_enc = x_mark_enc.to(self.device)
                x_dec = x_dec.to(self.device)
                x_mark_dec = x_mark_dec.to(self.device)
                y = y.to(self.device)

                # Permute dimensions to match model expectations
                x_enc = x_enc.permute(0, 2, 1)  # Shape: [batch_size, 1, seq_len]
                x_dec = x_dec.permute(0, 2, 1)  # Shape: [batch_size, 1, label_len + pred_len]
                y = y.permute(0, 2, 1)          # Shape: [batch_size, 1, pred_len]

                # Forward pass
                outputs = self.lora_model(
                    x_enc=x_enc,
                    x_mark_enc=x_mark_enc,
                    x_dec=x_dec,
                    x_mark_dec=x_mark_dec
                )
                loss = self._compute_loss(outputs, y)

                # Backward pass
                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()

            avg_loss = total_loss / len(train_loader)
            print(f"Epoch {epoch+1}/{self.num_epochs}, Loss: {avg_loss:.4f}")


    def _compute_loss(self, outputs, targets):
        """Compute appropriate loss based on task"""
        task_name = self.lora_model.base_model.configs.task_name

        if task_name in ["long_term_forecast", "short_term_forecast"]:
            return nn.MSELoss()(outputs, targets)
        elif task_name == "classification":
            return nn.CrossEntropyLoss()(outputs, targets)
        else:
            return nn.MSELoss()(outputs, targets)

    def save_lora_weights(self, path):
        """Save LoRA weights"""
        lora_state_dict = {
            name: param
            for name, param in self.lora_model.named_parameters()
            if "lora_" in name
        }
        torch.save(lora_state_dict, path)

    def load_lora_weights(self, path):
        """Load LoRA weights"""
        lora_state_dict = torch.load(path)
        self.lora_model.load_state_dict(lora_state_dict, strict=False)


In [5]:
import sys
sys.path.append('../src')

import torch
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from models.timesnet.timesnet import Model as TimesNet
from models.timesnet.lora_timesnet import LoRATimesNet
# from models.timesnet.lora_finetuner import LoRAFineTuner
from synth_data_generators.nonstationary import ARNonstationaryGenerator

# TimesNet configs
class Config:
    def __init__(self):
        self.seq_len = 900
        self.pred_len = 96
        self.label_len = 48
        self.task_name = 'long_term_forecast'
        self.enc_in = 1  # Set to 1 for univariate data
        self.dec_in = 1
        self.c_out = 1
        self.d_model = 16
        self.d_ff = 32
        self.num_kernels = 6
        self.top_k = 5
        self.e_layers = 2
        self.embed = 'timeF'
        self.freq = 'h'
        self.dropout = 0.1
        
# Initialize models
print("Initializing models...")
configs = Config()
base_model = TimesNet(configs)

# Initialize fine-tuner
fine_tuner = LoRAFineTuner(
    base_model=base_model,
    rank=4,
    alpha=1.0,
    learning_rate=1e-3,
    batch_size=32,
    num_epochs=10
)

# Create non-stationary data generator
generator = ARNonstationaryGenerator(
    seq_length=configs.seq_len + configs.pred_len,  # Now 900 + 96 = 996
    num_samples=1000,
    noise_variance=16
)

# Train on synthetic data
print("Training on synthetic non-stationary data...")
train_metrics = fine_tuner.train_on_synthetic(generator)

# Save model and configuration
save_path = 'nonstationary_lora_model.pt'
torch.save({
    'model_state_dict': fine_tuner.lora_model.state_dict(),
    'train_metrics': train_metrics,
    'configs': configs
}, save_path)
print(f"Model saved to {save_path}")

# Plot training metrics if available
if hasattr(train_metrics, 'loss'):
    plt.figure(figsize=(10, 5))
    plt.plot(train_metrics['loss'], label='Training Loss')
    plt.title('Training Progress')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

Initializing models...
Training on synthetic non-stationary data...


RuntimeError: Given groups=1, weight of size [16, 1, 3], expected input[32, 900, 3] to have 1 channels, but got 900 channels instead